In [4]:
import os
import pandas as pd


all_positions = pd.DataFrame()

OPTIONS_FOLDER = "database/options/"

def load_expiry_folders():
    return os.listdir(OPTIONS_FOLDER)

In [5]:
import pandas as pd

def get_nearest_expiry(expiry_folders, timestamp):
    nearest_expiry_folder = None
    for expiry_folder in expiry_folders:
        expiry_date = pd.to_datetime(expiry_folder)
        if expiry_date >= timestamp:
            if nearest_expiry_folder is None or expiry_date < nearest_expiry_folder:
                nearest_expiry_folder = expiry_date

    nearest_expiry_folder = (
        pd.to_datetime(nearest_expiry_folder).strftime("%d%b%y").upper()
    )

    return nearest_expiry_folder

In [6]:
from summary import calculate_stats_from_trades, generate_markdown_report

expiry_folders = load_expiry_folders()
positions_files = []


for root, dirs, files in os.walk("directional_results"):
    for file in files:
        if file.endswith("positions.csv"):
            positions_files.append(os.path.join(root, file))


non_empty_positions_files = [
    file for file in positions_files if os.path.getsize(file) > 2
]


if non_empty_positions_files:
    positions = pd.DataFrame()
    for file in non_empty_positions_files:
        print(f"Processing {file}...")
        positions_df = pd.read_csv(file)
        new_positions_df = pd.DataFrame()

        print(f"Found {len(positions_df)} positions...")

        for i in range(0, len(positions_df), 2):
            if i + 1 >= len(positions_df):
                continue

            row1 = positions_df.iloc[i]
            row2 = positions_df.iloc[i + 1]

            new_row = {}

            new_row["Instruments"] = (
                str(row1["Strike"])
                + str(row1["Option Type"])
                + ", "
                + str(row2["Strike"])
                + str(row2["Option Type"])
            )

            new_row["Entry Timestamp"] = min(
                row1["Entry Timestamp"], row2["Entry Timestamp"]
            )

            new_row["Entry Price"] = row1["Entry Price"] + row2["Entry Price"]

            new_row["Exit Timestamp"] = max(
                row1["Exit Timestamp"], row2["Exit Timestamp"]
            )

            new_row["Exit Price"] = row1["Exit Price"] + row2["Exit Price"]

            new_row["Quantity"] = row1["Quantity"] + row2["Quantity"]

            new_row["Lot Size"] = row1["Lot Size"]

            new_row["PnL per Lot"] = row1["PnL per Lot"] + row2["PnL per Lot"]

            new_row["Cost per Lot"] = row1["Cost per Lot"] + row2["Cost per Lot"]

            new_row["Net PnL per Lot"] = (
                row1["Net PnL per Lot"] + row2["Net PnL per Lot"]
            )

            new_row["Exit Reason"] = row1["Exit Reason"]

            new_row["Nearest Expiry Date"] = get_nearest_expiry(
                expiry_folders, pd.to_datetime(new_row["Exit Timestamp"].split(" ")[0])
            )

            new_row["Days to Expiry"] = (
                pd.to_datetime(new_row["Nearest Expiry Date"])
                - pd.to_datetime(new_row["Exit Timestamp"])
            ).days + 1

            new_row["Expiry Day Flag"] = new_row["Days to Expiry"] == 0

            new_row["Month"] = pd.to_datetime(new_row["Exit Timestamp"]).month

            new_row["Hold Time"] = max(row1["Hold Time"], row2["Hold Time"])

            new_positions_df = pd.concat([new_positions_df, pd.DataFrame([new_row])])

        print(f"Created {len(new_positions_df)} positions...")

        positions = pd.concat([positions, new_positions_df])

    print(f"Final positions count: {len(positions)}")

    positions = positions.sort_values(by="Entry Timestamp")


positions.to_csv("directional_results_combined_positions.csv", index=False)

positions["Entry Timestamp"] = positions["Entry Timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
positions["Exit Timestamp"] = positions["Exit Timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
positions["Expiry Day Flag"] = positions["Expiry Day Flag"].astype(bool)

stats = calculate_stats_from_trades(trades=positions)
stats = pd.DataFrame(stats, index=[0])
cols = stats.columns.tolist()

stats.to_csv("directional_results_combined_stats.csv", index=False)

generate_markdown_report(
    trades=positions,
    template_path="report_template.md",
    output_path="directional_results_combined_report.md",
)

Processing directional_results\2024-01-01_positions.csv...
Found 10 positions...
Created 5 positions...
Processing directional_results\2024-01-02_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-03_positions.csv...
Found 10 positions...
Created 5 positions...
Processing directional_results\2024-01-04_positions.csv...
Found 6 positions...
Created 3 positions...
Processing directional_results\2024-01-05_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-08_positions.csv...
Found 8 positions...
Created 4 positions...
Processing directional_results\2024-01-09_positions.csv...
Found 10 positions...
Created 5 positions...
Processing directional_results\2024-01-10_positions.csv...
Found 12 positions...
Created 6 positions...
Processing directional_results\2024-01-12_positions.csv...
Found 12 positions...
Created 6 positions...
Processing directional_results\2024-01-15_positions.csv...
Found 8 p

{'Trade Month': 'Unknown',
 'Max Capital Required': 72087.449625,
 'Return On Capital': np.float64(-87.89776237499997),
 'Total Cost': np.float64(39339.47475000001),
 'PnL': np.float64(-136456.04999999996),
 'Net PnL': np.float64(-175795.52474999995),
 'Win Rate': 0.3022861981371719,
 'Profit Factor': np.float64(0.7071531879992385),
 'Expiry Day PnL': np.float64(-59101.899825),
 'Average Return per Trade': np.float64(-148.85311155800164),
 'Total Trades': 1181,
 'Profitable Trades': 337,
 'Losing Trades': 844,
 'Average Duration': np.float64(39.53175275190517),
 'Average Winning Trade': np.float64(1259.6529763353117),
 'Average Losing Trade': np.float64(-711.2542390699051),
 'Largest Winning Trade': np.float64(12688.064400000003),
 'Largest Losing Trade': np.float64(-5843.877675),
 'Risk Reward Ratio': np.float64(1.7710305361167873),
 'CAGR': np.float64(-0.8789776237500009),
 'Calmar Ratio': np.float64(-0.9783264444959435),
 'Consecutive Wins': np.int64(5),
 'Consecutive Losses': np.in

In [7]:
all_positions = pd.concat([all_positions, positions])

In [8]:
import plotly.graph_objects as go

equity_curve = positions[["Exit Timestamp", "Net PnL per Lot"]].copy()
equity_curve["Cumulative PnL"] = equity_curve["Net PnL per Lot"].cumsum() + 200000

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=equity_curve["Exit Timestamp"],
        y=equity_curve["Cumulative PnL"],
        mode="lines",
        name="Equity Curve",
    )
)

fig.update_layout(
    title="Equity Curve",
    xaxis_title="Date",
    yaxis_title="Cumulative PnL",
    showlegend=True,
)

fig.write_html("directional_results_combined_equity_curve.html")

In [9]:
from summary import calculate_stats_from_trades, generate_markdown_report

expiry_folders = load_expiry_folders()

positions_files = []

for root, dirs, files in os.walk("semi_directional_results"):

    for file in files:

        if file.endswith("positions.csv"):

            positions_files.append(os.path.join(root, file))


non_empty_positions_files = [
    file for file in positions_files if os.path.getsize(file) > 2
]


if non_empty_positions_files:

    positions = pd.DataFrame()

    for file in non_empty_positions_files:

        print(f"Processing {file}...")

        positions_df = pd.read_csv(file)

        new_positions_df = pd.DataFrame()

        print(f"Found {len(positions_df)} positions...")

        for i in range(0, len(positions_df), 2):

            if i + 1 >= len(positions_df):

                continue

            row1 = positions_df.iloc[i]

            row2 = positions_df.iloc[i + 1]

            new_row = {}

            new_row["Instruments"] = (
                str(row1["Strike"])
                + str(row1["Option Type"])
                + ", "
                + str(row2["Strike"])
                + str(row2["Option Type"])
            )

            new_row["Entry Timestamp"] = min(
                row1["Entry Timestamp"], row2["Entry Timestamp"]
            )

            new_row["Entry Price"] = row1["Entry Price"] + row2["Entry Price"]

            new_row["Exit Timestamp"] = max(
                row1["Exit Timestamp"], row2["Exit Timestamp"]
            )

            new_row["Exit Price"] = row1["Exit Price"] + row2["Exit Price"]

            new_row["Quantity"] = row1["Quantity"] + row2["Quantity"]

            new_row["Lot Size"] = row1["Lot Size"]

            new_row["PnL per Lot"] = row1["PnL per Lot"] + row2["PnL per Lot"]

            new_row["Cost per Lot"] = row1["Cost per Lot"] + row2["Cost per Lot"]

            new_row["Net PnL per Lot"] = (
                row1["Net PnL per Lot"] + row2["Net PnL per Lot"]
            )

            new_row["Exit Reason"] = row1["Exit Reason"]

            new_row["Nearest Expiry Date"] = get_nearest_expiry(
                expiry_folders, pd.to_datetime(new_row["Exit Timestamp"].split(" ")[0])
            )

            new_row["Days to Expiry"] = (
                pd.to_datetime(new_row["Nearest Expiry Date"])
                - pd.to_datetime(new_row["Exit Timestamp"])
            ).days + 1

            new_row["Expiry Day Flag"] = new_row["Days to Expiry"] == 0
            
            new_row["Month"] = pd.to_datetime(new_row["Exit Timestamp"]).month

            new_row["Hold Time"] = max(row1["Hold Time"], row2["Hold Time"])

            new_positions_df = pd.concat([new_positions_df, pd.DataFrame([new_row])])

        print(f"Created {len(new_positions_df)} positions...")

        positions = pd.concat([positions, new_positions_df])

    print(f"Final positions count: {len(positions)}")

    positions = positions.sort_values(by="Entry Timestamp")


positions.to_csv("semi_directional_results_combined_positions.csv", index=False)

positions["Entry Timestamp"] = positions["Entry Timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
positions["Exit Timestamp"] = positions["Exit Timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
positions["Expiry Day Flag"] = positions["Expiry Day Flag"].astype(bool)

stats = calculate_stats_from_trades(trades=positions)
stats = pd.DataFrame(stats, index=[0])
cols = stats.columns.tolist()

stats.to_csv("semi_directional_results_combined_stats.csv", index=False)

generate_markdown_report(
    trades=positions,
    template_path="report_template.md",
    output_path="semi_directional_results_combined_report.md"
)

Processing semi_directional_results\2024-01-01_positions.csv...
Found 8 positions...
Created 4 positions...
Processing semi_directional_results\2024-01-02_positions.csv...
Found 4 positions...
Created 2 positions...
Processing semi_directional_results\2024-01-03_positions.csv...
Found 4 positions...
Created 2 positions...
Processing semi_directional_results\2024-01-04_positions.csv...
Found 2 positions...
Created 1 positions...
Processing semi_directional_results\2024-01-05_positions.csv...
Found 4 positions...
Created 2 positions...
Processing semi_directional_results\2024-01-08_positions.csv...
Found 2 positions...
Created 1 positions...
Processing semi_directional_results\2024-01-09_positions.csv...
Found 6 positions...
Created 3 positions...
Processing semi_directional_results\2024-01-10_positions.csv...
Found 12 positions...
Created 6 positions...
Processing semi_directional_results\2024-01-12_positions.csv...
Found 10 positions...
Created 5 positions...
Processing semi_directiona

{'Trade Month': 'Unknown',
 'Max Capital Required': 63996.349725,
 'Return On Capital': np.float64(-15.235692187499989),
 'Total Cost': np.float64(21608.634375),
 'PnL': np.float64(-8862.749999999982),
 'Net PnL': np.float64(-30471.38437499998),
 'Win Rate': 0.3128205128205128,
 'Profit Factor': np.float64(0.9244315323805846),
 'Expiry Day PnL': np.float64(-5767.764074999996),
 'Average Return per Trade': np.float64(-39.06587740384613),
 'Total Trades': 780,
 'Profitable Trades': 241,
 'Losing Trades': 539,
 'Average Duration': np.float64(51.991025641025644),
 'Average Winning Trade': np.float64(1546.7113702282159),
 'Average Losing Trade': np.float64(-748.105425974026),
 'Largest Winning Trade': np.float64(11866.040550000005),
 'Largest Losing Trade': np.float64(-2511.8700000000017),
 'Risk Reward Ratio': np.float64(2.067504547523382),
 'CAGR': np.float64(-0.15235692187499983),
 'Calmar Ratio': np.float64(-0.6859702041130208),
 'Consecutive Wins': np.int64(6),
 'Consecutive Losses': n

In [10]:
import plotly.graph_objects as go

equity_curve = positions[["Exit Timestamp", "Net PnL per Lot"]].copy()
equity_curve["Cumulative PnL"] = equity_curve["Net PnL per Lot"].cumsum() + 200000

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=equity_curve["Exit Timestamp"],
        y=equity_curve["Cumulative PnL"],
        mode="lines",
        name="Equity Curve",
    )
)

fig.update_layout(
    title="Equity Curve",
    xaxis_title="Date",
    yaxis_title="Cumulative PnL",
    showlegend=True,
)

fig.write_html("semi_directional_results_combined_equity_curve.html")

In [11]:
all_positions = pd.concat([all_positions, positions])

In [12]:

from summary import calculate_stats_from_trades, generate_markdown_report

expiry_folders = load_expiry_folders()

positions_files = []

for root, dirs, files in os.walk("mean_reversion_results"):

    for file in files:

        if file.endswith("positions.csv"):

            positions_files.append(os.path.join(root, file))


non_empty_positions_files = [
    file for file in positions_files if os.path.getsize(file) > 2
]


if non_empty_positions_files:

    positions = pd.DataFrame()

    for file in non_empty_positions_files:

        print(f"Processing {file}...")

        positions_df = pd.read_csv(file)

        new_positions_df = pd.DataFrame()

        print(f"Found {len(positions_df)} positions...")

        for i in range(0, len(positions_df), 2):

            if i + 1 >= len(positions_df):

                continue

            row1 = positions_df.iloc[i]

            row2 = positions_df.iloc[i + 1]

            new_row = {}

            new_row["Instruments"] = (
                str(row1["Strike"])
                + str(row1["Option Type"])
                + ", "
                + str(row2["Strike"])
                + str(row2["Option Type"])
            )

            new_row["Entry Timestamp"] = min(
                row1["Entry Timestamp"], row2["Entry Timestamp"]
            )

            new_row["Entry Price"] = row1["Entry Price"] + row2["Entry Price"]

            new_row["Exit Timestamp"] = max(
                row1["Exit Timestamp"], row2["Exit Timestamp"]
            )

            new_row["Exit Price"] = row1["Exit Price"] + row2["Exit Price"]

            new_row["Quantity"] = row1["Quantity"] + row2["Quantity"]

            new_row["Lot Size"] = row1["Lot Size"]

            new_row["PnL per Lot"] = row1["PnL per Lot"] + row2["PnL per Lot"]

            new_row["Cost per Lot"] = row1["Cost per Lot"] + row2["Cost per Lot"]

            new_row["Net PnL per Lot"] = (
                row1["Net PnL per Lot"] + row2["Net PnL per Lot"]
            )

            new_row["Exit Reason"] = row1["Exit Reason"]

            new_row["Nearest Expiry Date"] = get_nearest_expiry(
                expiry_folders, pd.to_datetime(new_row["Exit Timestamp"].split(" ")[0])
            )

            new_row["Days to Expiry"] = (
                pd.to_datetime(new_row["Nearest Expiry Date"])
                - pd.to_datetime(new_row["Exit Timestamp"])
            ).days + 1

            new_row["Expiry Day Flag"] = new_row["Days to Expiry"] == 0

            new_row["Month"] = pd.to_datetime(new_row["Exit Timestamp"]).month

            new_row["Hold Time"] = max(row1["Hold Time"], row2["Hold Time"])

            new_positions_df = pd.concat([new_positions_df, pd.DataFrame([new_row])])

        print(f"Created {len(new_positions_df)} positions...")

        positions = pd.concat([positions, new_positions_df])

    print(f"Final positions count: {len(positions)}")

    positions = positions.sort_values(by="Entry Timestamp")


positions.to_csv("mean_reversion_results_combined_positions.csv", index=False)

positions["Entry Timestamp"] = positions["Entry Timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
positions["Exit Timestamp"] = positions["Exit Timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
positions["Expiry Day Flag"] = positions["Expiry Day Flag"].astype(bool)

stats = calculate_stats_from_trades(trades=positions)
stats = pd.DataFrame(stats, index=[0])
cols = stats.columns.tolist()

stats.to_csv("mean_reversion_results_combined_stats.csv", index=False)

generate_markdown_report(
    trades=positions,
    template_path="report_template.md",
    output_path="mean_reversion_results_combined_report.md",
)

Processing mean_reversion_results\2024-01-01_positions.csv...
Found 4 positions...
Created 2 positions...
Processing mean_reversion_results\2024-01-02_positions.csv...
Found 4 positions...
Created 2 positions...
Processing mean_reversion_results\2024-01-03_positions.csv...
Found 4 positions...
Created 2 positions...
Processing mean_reversion_results\2024-01-04_positions.csv...
Found 2 positions...
Created 1 positions...
Processing mean_reversion_results\2024-01-05_positions.csv...
Found 4 positions...
Created 2 positions...
Processing mean_reversion_results\2024-01-08_positions.csv...
Found 6 positions...
Created 3 positions...
Processing mean_reversion_results\2024-01-09_positions.csv...
Found 4 positions...
Created 2 positions...
Processing mean_reversion_results\2024-01-10_positions.csv...
Found 6 positions...
Created 3 positions...
Processing mean_reversion_results\2024-01-12_positions.csv...
Found 6 positions...
Created 3 positions...
Processing mean_reversion_results\2024-01-15_p

{'Trade Month': 'Unknown',
 'Max Capital Required': 66117.79665,
 'Return On Capital': np.float64(0.7816642500000035),
 'Total Cost': np.float64(16802.259),
 'PnL': np.float64(18365.587500000005),
 'Net PnL': np.float64(1563.328500000007),
 'Win Rate': 0.6507936507936508,
 'Profit Factor': np.float64(1.0082739517532957),
 'Expiry Day PnL': np.float64(2201.5298250000055),
 'Average Return per Trade': np.float64(3.1018422619047756),
 'Total Trades': 504,
 'Profitable Trades': 317,
 'Losing Trades': 187,
 'Average Duration': np.float64(34.04563492063492),
 'Average Winning Trade': np.float64(600.9751942429023),
 'Average Losing Trade': np.float64(-1010.4053907754011),
 'Largest Winning Trade': np.float64(5437.637999999997),
 'Largest Losing Trade': np.float64(-11896.893525),
 'Risk Reward Ratio': np.float64(0.5947862112866444),
 'CAGR': np.float64(0.007816642499999915),
 'Calmar Ratio': np.float64(0.06825073768246581),
 'Consecutive Wins': np.int64(10),
 'Consecutive Losses': np.int64(6),

In [13]:
all_positions = pd.concat([all_positions, positions])
all_positions = all_positions.sort_values(by="Entry Timestamp")
all_positions.to_csv("portfolio_combined_positions.csv", index=False)

In [14]:
import plotly.graph_objects as go

equity_curve = positions[["Exit Timestamp", "Net PnL per Lot"]].copy()
equity_curve["Cumulative PnL"] = equity_curve["Net PnL per Lot"].cumsum() + 200000

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=equity_curve["Exit Timestamp"],
        y=equity_curve["Cumulative PnL"],
        mode="lines",
        name="Equity Curve",
    )
)

fig.update_layout(
    title="Equity Curve",
    xaxis_title="Date",
    yaxis_title="Cumulative PnL",
    showlegend=True,
)

fig.write_html("mean_reversion_results_combined_equity_curve.html")

In [15]:
stats = calculate_stats_from_trades(trades=all_positions, starting_capital=400000)
stats = pd.DataFrame(stats, index=[0])
cols = stats.columns.tolist()

stats.to_csv("portfolio_results_combined_stats.csv", index=False)

generate_markdown_report(
    trades=all_positions,
    template_path="report_template.md",
    output_path="portfolio_results_combined_report.md",
    starting_capital=400000,
)

TypeError: calculate_stats_from_trades() got an unexpected keyword argument 'starting_capital'

In [ ]:
import plotly.graph_objects as go

equity_curve = all_positions[["Exit Timestamp", "Net PnL per Lot"]].copy()
equity_curve["Cumulative PnL"] = equity_curve["Net PnL per Lot"].cumsum() + 400000

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=equity_curve["Exit Timestamp"],
        y=equity_curve["Cumulative PnL"],
        mode="lines",
        name="Equity Curve",
    )
)

fig.update_layout(
    title="Equity Curve",
    xaxis_title="Date",
    yaxis_title="Cumulative PnL",
    showlegend=True,
)

fig.write_html("portfolio_results_combined_equity_curve.html")